## model training data

In [44]:
import nltk 
import string 
import re 
import pandas as pd
import boto3
import pickle

In [23]:
#import data

In [24]:
df_inroads = pd.read_csv('https://inroads-test-bucket1.s3.amazonaws.com/inroads_only.csv', dtype={'text': str})
#df_raw = pd.read_csv('https://inroads-test-bucket1.s3.amazonaws.com/raw_data.csv', dtype={'text': str})

In [45]:
text_to_clean = df_inroads

In [ ]:
# preprocessing functions

In [46]:
def text_lowercase(text): 
    return text.lower() 

In [27]:
def remove_numbers(text): 
    result = re.sub(r'\d+', '', text) 
    return result 

In [28]:
def remove_punctuation(text):
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    temp = text.translate(translator)
    return re.sub('\s+',' ', temp)

In [29]:
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
lemmatizer = WordNetLemmatizer() 
nltk.download("punkt")
nltk.download('wordnet')
# lemmatize string 

def lemmatize_word(text): 
    word_tokens = word_tokenize(text) 
    # provide context i.e. part-of-speech 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens] 
    return lemmas 

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
def remove_short_words(text):
    clean_words = []
    for word in text:
        new = re.sub(r'\b\w{,3}\b', '', word)
        clean_words.append(new) 
    return clean_words

In [47]:
result = list()

for i in range(0,len(text_to_clean)):
    temp = str(text_to_clean.text.iloc[i])     
    temp = text_lowercase(temp) #make all lower
    temp_2 = remove_numbers(temp) # remove all numbers
    temp_3 = str.strip(temp_2) # remove white spaces 1
    temp_4 = temp_3.replace('\n',' ').replace("\t", " ") # remove white spaces 2
    temp_5 = re.sub(" +", " ", temp_4) # remove white spaces 3
    temp_6 = remove_punctuation(temp_5) # remove punctuation 
    temp_7 = lemmatize_word(temp_6) # lemmatize text
    temp_8 = [word for word in temp_7 if word.isalnum()] # remove auxiliarly punctuation
    temp_9 = remove_short_words(temp_8) # remove short words
    result.append(list(filter(None, temp_9))) # remove empty strings and save

In [48]:
text_to_clean['clean_text'] = result

In [199]:
## review

In [50]:
text_to_clean.head

<bound method NDFrame.head of       Unnamed: 0                                           resource  \
0              1  649607\nresearch-article2016\n                ...   
1              2  649607\nresearch-article2016\n                ...   
2              3  649607\nresearch-article2016\n                ...   
3              4  649607\nresearch-article2016\n                ...   
4              5  649607\nresearch-article2016\n                ...   
...          ...                                                ...   
1221        1222  the-inroads-team-is-growing-introducing-our-ne...   
1222        1223  there-is-no-qualification-to-be-here-just-pass...   
1223        1224                                           wfac.txt   
1224        1225  what-is-inspiring-our-steering-committee-in-20...   
1225        1226  young-people-fight-societal-norms-in-sudan-whe...   

                                                   text  \
0     649607\nresearch-article2016\n                ...   

In [51]:
text_to_clean.clean_text[:100]

0     [research, article, pspxxx, personality, socia...
1     [huang, describe, current, program, research, ...
2     [personality, social, psychology, bulletin, mo...
3     [huang, abortion, attitudes, over, year, perio...
4     [table, descriptive, statistics, bivariate, co...
                            ...                        
95    [towards, asps, show, figure, informants, expr...
96    [behavior, community, towards, people, associa...
97    [abortion, brother, stop, talk, with, after, l...
98    [person, support, abortion, spaw, fgds, partic...
99    [women, exclude, religious, practice, abortive...
Name: clean_text, Length: 100, dtype: object

In [202]:
## save cleaned tokens for further use

In [52]:
final_inroads_clean = text_to_clean[['resource', 'clean_text']].copy()

In [53]:
final_inroads_clean.head

<bound method NDFrame.head of                                                resource  \
0     649607\nresearch-article2016\n                ...   
1     649607\nresearch-article2016\n                ...   
2     649607\nresearch-article2016\n                ...   
3     649607\nresearch-article2016\n                ...   
4     649607\nresearch-article2016\n                ...   
...                                                 ...   
1221  the-inroads-team-is-growing-introducing-our-ne...   
1222  there-is-no-qualification-to-be-here-just-pass...   
1223                                           wfac.txt   
1224  what-is-inspiring-our-steering-committee-in-20...   
1225  young-people-fight-societal-norms-in-sudan-whe...   

                                             clean_text  
0     [research, article, pspxxx, personality, socia...  
1     [huang, describe, current, program, research, ...  
2     [personality, social, psychology, bulletin, mo...  
3     [huang, abortion, attit

In [54]:
final_inroads_clean.to_pickle('final_inroads_clean.pkl')
final_inroads_clean = pd.read_pickle('final_inroads_clean.pkl')

In [55]:
# Create an empty list 
temp_list =[] 

# Iterate over each row 
for index, rows in final_inroads_clean.iterrows(): 
    # Create list for the current row 
    my_list =[rows.clean_text] 

    # append the list to the final list 
    temp_list.append(my_list) 

# Print the list 
print(temp_list[0]) 


[['research', 'article', 'pspxxx', 'personality', 'social', 'psychology', 'bulletinhuang', 'article', 'personality', 'social', 'benevolent', 'sexism', 'attitudes', 'toward', 'psychology', 'bulletin', 'society', 'personality', 'motherhood', 'reproductive', 'right', 'social', 'psychology', 'reprint', 'permissions', 'multi', 'study', 'longitudinal', 'examination', 'sagepub', 'journalspermissions', 'abortion', 'attitudes', 'pspb', 'sagepub', 'yanshu', 'huang', 'paul', 'davies', 'chris', 'sibley', 'danny', 'osborne', 'abstract', 'although', 'benevolent', 'sexism', 'ideology', 'that', 'highly', 'revere', 'women', 'conform', 'traditional', 'gender', 'cloak', 'superficially', 'positive', 'tone', 'place', 'upon', 'pedestal', 'inherently', 'restrictive', 'accordingly', 'because', 'paternalistic', 'beliefs', 'associate', 'with', 'base', 'idealization', 'traditional', 'gender', 'roles', 'which', 'include', 'motherhood', 'should', 'predict', 'people', 'attitudes', 'toward', 'women', 'reproductive',

In [57]:
len(temp_list)

1226

In [58]:
clean_inroads_tokens = []

for i in range(0,len(final_inroads_clean)):
    text = final_inroads_clean.clean_text[i]
    #print(text)
    clean_inroads_tokens.extend(text)

In [59]:
clean_inroads_tokens = pd.DataFrame(clean_inroads_tokens,columns=['tokens'])
clean_inroads_tokens.head

<bound method NDFrame.head of              tokens
0          research
1           article
2            pspxxx
3       personality
4            social
...             ...
321033       slowly
321034        there
321035        space
321036        start
321037   discussion

[321038 rows x 1 columns]>

In [60]:
clean_inroads_tokens.to_pickle('clean_inroads_tokens.pkl')

In [61]:
clean_inroads_tokens = pd.read_pickle('clean_inroads_tokens.pkl')

In [212]:
## upload trainign data

In [213]:
import boto3
bucket = 'sagemaker-word2vec-scikitlearn'
region = 'us-east-2'
s3_session = boto3.Session().resource('s3')
s3_session.create_bucket(Bucket=bucket, 
                         CreateBucketConfiguration=
                         {'LocationConstraint': region})
s3_session.Bucket(bucket).Object('train/clean_full_tokens.pkl').upload_file('clean_full_tokens.pkl')

BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

In [214]:
## create dictionary

In [215]:
word_dict = clean_inroads_tokens.to_dict() 

In [216]:
word_dict

{'tokens': {0: 'research',
  1: 'article',
  2: 'pspxxx',
  3: 'personality',
  4: 'social',
  5: 'psychology',
  6: 'bulletinhuang',
  7: 'article',
  8: 'personality',
  9: 'social',
  10: 'benevolent',
  11: 'sexism',
  12: 'attitudes',
  13: 'toward',
  14: 'psychology',
  15: 'bulletin',
  16: 'society',
  17: 'personality',
  18: 'motherhood',
  19: 'reproductive',
  20: 'right',
  21: 'social',
  22: 'psychology',
  23: 'reprint',
  24: 'permissions',
  25: 'multi',
  26: 'study',
  27: 'longitudinal',
  28: 'examination',
  29: 'sagepub',
  30: 'journalspermissions',
  31: 'abortion',
  32: 'attitudes',
  33: 'pspb',
  34: 'sagepub',
  35: 'yanshu',
  36: 'huang',
  37: 'paul',
  38: 'davies',
  39: 'chris',
  40: 'sibley',
  41: 'danny',
  42: 'osborne',
  43: 'abstract',
  44: 'although',
  45: 'benevolent',
  46: 'sexism',
  47: 'ideology',
  48: 'that',
  49: 'highly',
  50: 'revere',
  51: 'women',
  52: 'conform',
  53: 'traditional',
  54: 'gender',
  55: 'cloak',
  56: 

In [64]:
data_dir = 'dict_data/word2vec' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [71]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

## old user data for training

In [319]:
old_data = pd.read_csv('https://inroads-test-bucket1.s3.amazonaws.com/Taxonomy-Grid+view+(1).csv', dtype={'text': str})

In [320]:
old_data.head

<bound method NDFrame.head of                RecordID       Term      Domain      Subdomain
0     rec8EgTumvHTEJdLa   abstract  Conceptual     Conceptual
1     recMJFlgfo00roAuw    absence  Conceptual     Conceptual
2     recAuJIeX88OP6ZIb  abundance  Conceptual     Conceptual
3     recz0CPCI09cP5iUX    reality  Conceptual     Conceptual
4     rec8Bm0aJwpSKdOGQ  beautiful  Conceptual     Conceptual
...                 ...        ...         ...            ...
1016  recMKWuPq8dMo70HD     summer    Temporal  Units of time
1017  recjrhx9mX0O1xCu0     autumn    Temporal  Units of time
1018  recraEJIJ7Fe5acU5       1971    Temporal  Units of time
1019  recvyH9ENCss8n1xG       1972    Temporal  Units of time
1020  recKH4SX6UXD9wvKv        NaN         NaN            NaN

[1021 rows x 4 columns]>

In [321]:
old_data = old_data[['Term','Domain']].copy().sort_values(['Term','Domain'])

In [14]:
old_data.to_csv('inroads_old_data.csv')
old_data.to_pickle('inroads_old.pkl')

In [15]:
## clean

In [322]:
result = list()

for i in range(0,len(old_data)):
    temp = str(old_data.Term.iloc[i])     
    temp = text_lowercase(temp) #make all lower
    temp_2 = remove_numbers(temp) # remove all numbers
    temp_3 = str.strip(temp_2) # remove white spaces 1
    temp_4 = temp_3.replace('\n',' ').replace("\t", " ") # remove white spaces 2
    temp_5 = re.sub(" +", " ", temp_4) # remove white spaces 3
    temp_6 = remove_punctuation(temp_5) # remove punctuation 
    temp_7 = lemmatize_word(temp_6) # lemmatize text
    temp_8 = [word for word in temp_7 if word.isalnum()] # remove auxiliarly punctuation
    temp_9 = remove_short_words(temp_8) # remove short words
    result.append(list(filter(None, temp_9))) # remove empty strings and save

In [323]:
old_data['clean_text'] = result

In [324]:
old_data.clean_text.head()

724              []
1018             []
1019             []
928     [september]
682              []
Name: clean_text, dtype: object

In [325]:
old_tokens = []

for i in range(0,len(old_data)):
    text = old_data.clean_text[i]
    #print(text)
    old_tokens.extend(text)

In [326]:
old_tokens = pd.DataFrame(old_tokens,columns=['tokens'])
old_tokens.head

<bound method NDFrame.head of          tokens
0      abstract
1       absence
2     abundance
3       reality
4     beautiful
...         ...
1307     winter
1308     spring
1309     season
1310     summer
1311     autumn

[1312 rows x 1 columns]>

In [69]:
old_tokens.to_pickle('old_tokens.pkl')

## train word2vec


In [117]:
!pip install gensim
from gensim.models.word2vec import Word2Vec
from sklearn.cluster import KMeans;
#from sklearn.neighbors import KDTree
import numpy as np

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [327]:
token_list = list(final_inroads_clean.clean_text)

you can use the same model I use, which is saved, I provide code to download it from s3.  Or, you can create your own by uncommenting the training code.

In [119]:
#num_features = 300
#min_word_count = 50
#num_workers = 2
#window_size = 6
#subsampling = 1e-3

In [122]:
#model = Word2Vec(
#    token_list, 
#    workers = num_workers, 
#    size = num_features, 
#    min_count=min_word_count, 
#    window=window_size, 
#    sample=subsampling)

we need both pieces to make the model work, I'm struggling to find a way for me to upload and download the gensim model from S3.  TO BE DETERMINED

In [195]:
import boto
from boto.s3.connection import OrdinaryCallingFormat

S3_BUCKET = 'inroads-test-bucket1'
S3_KEY = 'w2v_model.bin'

def download_gensim_model():
    s3_conn = boto3.client('s3', aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
                    aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'))
    bucket = s3_conn.get_bucket(S3_BUCKET)
    key = bucket.get_key(S3_KEY)
    model = gensim.models.Word2Vec.load_word2vec_format(key, binary=True)
    return model

model = download_gensim_model()

AttributeError: 'S3' object has no attribute 'get_bucket'

In [328]:
model_name = "inroads_word2vec_model"
model = Word2Vec.load(model_name)

In [329]:
model.wv.most_similar("abortion")

[('abortions', 0.5601829290390015),
 ('postabortion', 0.4012739658355713),
 ('contraception', 0.3701522946357727),
 ('abor', 0.36626535654067993),
 ('medically', 0.34925901889801025),
 ('antenatal', 0.3442828059196472),
 ('termination', 0.3441293239593506),
 ('reproductive', 0.3407558798789978),
 ('misoprostol', 0.338686466217041),
 ('contraceptive', 0.33505553007125854)]

In [330]:
def get_vector(text):
    vector_list = []
    
    for i in range(len(text)):
        word = text[i]     
        try: 
            vector = model.wv[word]
        except KeyError:
            pass
        else:
            if vector is None:
                vector = np.zeros(300,dtype=int) 
                vector_list.append([word, vector])
            else: 
                vector_list.append([word, vector])
        #vector_embed.append(pd.Series([word, vector], index=['token',range(0,300)]), ignore_index=True)
                    
    return vector_list 

In [331]:
token_list_old = list(old_tokens.tokens)
token_list_old = list(dict.fromkeys(token_list_old))
len(token_list_old)

982

In [332]:
old_vectors = get_vector(token_list_old)

In [333]:
old_vocab = list()

for i in range(0,len(old_vectors)):
    test = old_vectors[i]
    old_vocab.append(test[0])

In [334]:
old_vocab_df = pd.DataFrame(old_vocab, columns=["token"])

In [335]:
old_vectors_2 = list()

for i in range(0,len(old_vectors)):
    test = old_vectors[i]
    old_vectors_2.append(test[1])
    
old_array = np.array(old_vectors_2)
old_vectors_df = pd.DataFrame(old_vectors, columns=["token", "vector"])

In [395]:
old_vectors_df_2 = pd.DataFrame(old_vectors_df.vector.tolist())
old_vectors_df_2.head

result = pd.concat([old_vectors_df, old_vectors_df_2], axis=1)
result.head

<bound method NDFrame.head of          token                                             vector         0  \
0     abstract  [0.009340104, -0.011540192, -0.005010838, 0.17...  0.009340   
1      absence  [0.07312082, -0.047603212, 0.091147356, 0.1621...  0.073121   
2    abundance  [0.013253822, 0.05022224, -0.08944513, 0.08348...  0.013254   
3      reality  [0.017231788, -0.13826157, -0.06955002, 0.1615...  0.017232   
4    beautiful  [-0.056970224, 0.02030195, -0.024677064, -0.04... -0.056970   
..         ...                                                ...       ...   
936     winter  [-0.021195991, -0.011310648, -0.07302275, 0.03... -0.021196   
937     spring  [-0.028904684, 0.093232386, -0.058348857, -0.0... -0.028905   
938     season  [-0.074280344, 0.093354344, -0.14504637, 0.006... -0.074280   
939     summer  [-0.04597636, -0.006390778, -0.056196004, -0.0... -0.045976   
940     autumn  [0.014605279, 0.1677676, -0.08277614, -0.01768...  0.014605   

            1        

## Cluster

In [131]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering.cluster_centers_, idx;

In [132]:
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);
#Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];
#Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
#A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    return df;

In [133]:
centers,clusters = clustering_on_wordvecs(old_vectors_2, 9);
centroid_map = dict(zip(old_vocab, clusters));

In [226]:
centroid_map_df = pd.DataFrame(list(centroid_map.items()),columns = ['token','cluster']) 
centroid_map_df

,token,cluster
0,abstract,0 absence 1 abundance 2 ...
1,0,0 0 1 6 2 0 3 4 4 0 ...


## import saved cluster structures

In [356]:
centers = np.load("cluster_9_centers.npy")
clusters = np.load("cluster_9_clusters.npy")
centroid_map = pd.read_csv("cluster_9.csv", header=None)
centroid_map.columns = ["token", "cluster"]

In [359]:
centroid_map

,token,cluster
0,abstract,0
1,absence,0
2,abundance,6
3,reality,0
4,beautiful,4
...,...,...
936,winter,1
937,spring,1
938,season,1
939,summer,1


In [339]:
from sklearn.neighbors import KDTree

top_words = get_top_words(old_vocab, 20, centers, old_array)
print(top_words)

        Cluster #0 Cluster #1       Cluster #2 Cluster #3   Cluster #4  \
1            value       legs         violence       june         wife   
2            moral       feet           sexual   december       cousin   
3       individual     yellow           stigma    october      husband   
4           ethics      bleed             rape      april         aunt   
5        knowledge   sidewalk   discrimination    morning     daughter   
6          concept      sleep  criminalization       july      brother   
7           social    stomach            women     august       father   
8         morality    abdomen       stigmatize       week  grandmother   
9          process     toilet            abuse     summer    boyfriend   
10        autonomy    garbage   stigmatization   february       sister   
11           norms     purple             harm    january         girl   
12           human      stick         children      night        uncle   
13            need    scissor      pro

In [267]:
## merge vectors with clusters

In [396]:
result = pd.merge(result, centroid_map, how='left', on=['token'])
training_data = pd.merge(old_vocab_df, centroid_map, how='left', on=['token'])
training_data.to_csv('/home/ec2-user/SageMaker/deploy/training_data.csv',index=False)

result['cluster'] = result.cluster.astype(str)
result.dtypes 
result.head()

token       object
vector      object
0          float64
1          float64
2          float64
            ...   
296        float64
297        float64
298        float64
299        float64
cluster     object
Length: 303, dtype: object

In [12]:
import boto3
bucket = 'sagemaker-word2vec-scikitlearn'
region = 'us-east-2'
s3_session = boto3.Session().resource('s3')
s3_session.Bucket(bucket).Object('train/train.csv').upload_file('cluster_9.csv')

In [13]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

data_location = 'https://sagemaker-word2vec-scikitlearn.s3.us-east-2.amazonaws.com/train/train.csv'

train_data = pd.read_csv(data_location, header=None, names=None)

HTTPError: HTTP Error 403: Forbidden

## classification

In [397]:
result = result.dropna()
y = result.cluster
X = result.iloc[:, 2:302]

In [402]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.predict(X_test)

clf.predict_proba(X_test)

clf.score(X_test, y_test)


0.6779661016949152

In [398]:
from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.naive_bayes import MultinomialNB
from sklearn import svm


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

lin_clf = svm.LinearSVC()
clf = lin_clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [399]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test,y_pred)*100
accuracy

66.94915254237289

given the two relative accuracies it appears that logistic regression is just as powerful as svm, thus we can continue with logistic regression for ease

In [410]:
!cd sagemaker_inroads